In [31]:
# MMA Fight Network Dashboard (Voilà-ready)

import pandas as pd
import networkx as nx
from ipysigma import Sigma
import kagglehub
from kagglehub import KaggleDatasetAdapter
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

In [32]:
# Load dataset from Kaggle
file_path = "pro_mma_fights.csv"
df = kagglehub.load_dataset(
    KaggleDatasetAdapter.PANDAS,
    "binduvr/pro-mma-fights",
    file_path,
)

/var/folders/st/yx5_mdvj4bdbm59sg4_7n26m0000gq/T/ipykernel_1100/131653210.py:3: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  df = kagglehub.load_dataset(


In [33]:
# Clean the data
def clean_data(df):
    for col in ["fighter1_name", "fighter2_name"]:
        df[col] = df[col].str.strip().str.title()
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df["fight_id"] = (
        df["fighter1_name"] + "_vs_" + df["fighter2_name"] + "_" + df["date"].dt.strftime('%Y-%m-%d')
    ).fillna("unknown")
    return df

df = clean_data(df)

In [34]:
# Optional filter to just UFC
df = df[df['organisation'] == "Ultimate Fighting Championship (UFC)"]

In [35]:
# Build the graph
def build_graph(filtered_df):
    G = nx.Graph()
    for _, row in filtered_df.iterrows():
        f1 = row["fighter1_name"]
        f2 = row["fighter2_name"]
        result = row["fighter1_result"]
        method = row["win_method"]
        G.add_node(f1)
        G.add_node(f2)
        G.add_edge(f1, f2,
                   result=result,
                   method=method,
                   date=row["date"].strftime('%Y-%m-%d'),
                   event=row["event_title"])
    return G


In [36]:
# Widget: Select year
min_year = df["date"].dt.year.min()
max_year = df["date"].dt.year.max()
year_selector = widgets.IntSlider(
    value=max_year,
    min=min_year,
    max=max_year,
    step=1,
    description='Year:',
    continuous_update=False
)

output = widgets.Output()


In [ ]:
# Update function with centrality summary
def update_graph(year):
    output.clear_output()
    with output:
        filtered_df = df[df['date'].dt.year >= year]
        G = build_graph(filtered_df)
        G = G.subgraph([n for n in G.nodes if G.degree[n] >= 2]).copy()

        # Compute centrality scores
        degree = nx.degree_centrality(G)
        betweenness = nx.betweenness_centrality(G)
        eigenvector = nx.eigenvector_centrality(G, max_iter=500)

        # Find top nodes
        top_degree = sorted(degree.items(), key=lambda x: x[1], reverse=True)[:5]
        top_betweenness = sorted(betweenness.items(), key=lambda x: x[1], reverse=True)[:5]
        top_eigen = sorted(eigenvector.items(), key=lambda x: x[1], reverse=True)[:5]

        # Display summary in Markdown
        display(Markdown("### Top 5 Fighters by Degree Centrality"))
        for name, score in top_degree:
            display(Markdown(f"- {name}: {score:.4f}"))

        display(Markdown("### Top 5 Fighters by Betweenness Centrality"))
        for name, score in top_betweenness:
            display(Markdown(f"- {name}: {score:.4f}"))

        display(Markdown("### Top 5 Fighters by Eigenvector Centrality"))
        for name, score in top_eigen:
            display(Markdown(f"- {name}: {score:.4f}"))

        print(f"Fighters: {G.number_of_nodes()}, Fights: {G.number_of_edges()}")
        node_color = {node: G.degree[node] for node in G.nodes}
        node_size = degree
        display(Sigma(G, node_color=node_color, node_size=node_size))

In [41]:
# Link the widget to the update function
widgets.interact(update_graph, year=year_selector)
display(output)

interactive(children=(IntSlider(value=1993, continuous_update=False, description='Year:', max=2021, min=1993),…

Output(outputs=({'traceback': ['\x1b--------------------------------------------------------------------…